<a href="https://colab.research.google.com/github/kckiran01/CSC550/blob/master/gender_classification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir='/content/drive/MyDrive/csc591_avg/gender_class_avg/train_swipeLeft'
test_dir='/content/drive/MyDrive/csc591_avg/gender_class_avg/test_swipeLeft'

The class_mode should be binary since we will have two classes for gender classification

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#All images will be rescaled by 1./255 which may not necessary in our case though...Kiran

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir, # This is the cource directory for training images
    target_size=(256,256), # All images will be resized..not sure if this is necessary--- Kiran
    batch_size = 20,
    class_mode='categorical')

# Flow validation images in batches of 8 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (256,256),
    batch_size=10,
    shuffle=False,
    class_mode='binary')

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(256,256,3)),
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(20, activation = 'relu'),
                                    tf.keras.layers.Dense(30, activation='softmax')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.summary()
model.compile(
    loss='categorical_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=20, #total images = batch_size * steps--kiran
    epochs=30,
    validation_data=validation_generator,
    validation_steps=20 # total images = batch_size * steps--kiran
)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, recall_score, accuracy_score, precision_score
Y_pred = model.predict_generator(validation_generator,2) # class is 2
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(Y_pred)
print('y pred >>> git ')
print(y_pred)
#print(confusion_matrix(validation_generator.classes, y_pred))
print('classfication report')
print(str(np.mean(recall_score(validation_generator.classes, y_pred, average=None))))
accuracy =(accuracy_score(validation_generator.classes, y_pred))*100
print(str(accuracy))
prec= np.mean(precision_score(validation_generator.classes, y_pred, average=None))*100
print(str(prec))